In [1]:
import pandas as pd
import numpy as np

In [64]:
df_quotes=pd.read_csv('trades_USDC_OKEX.csv')

In [65]:
# Ensure sysTimestamp is in datetime format
df_quotes['sysTimestamp'] = pd.to_datetime(df_quotes['sysTimestamp'])

# Filter data for the first hour
first_hour_data = df_quotes[df_quotes['sysTimestamp'] < df_quotes['sysTimestamp'].iloc[0] + pd.Timedelta(hours=0.5)]

In [66]:
first_hour_data.to_csv('trades_USDC_O.csv', index=False)

In [ ]:
import json
import pandas as pd
import os
def flatten_trade(record):
    """Flatten a trade (T) message."""
    return {
        "exp": record.get("exp"),
        "sysTimestamp": record.get("header", {}).get("sysTimestamp"),
        "side": record.get("side"),
        "price": record.get("p"),
        "quantity": record.get("q"),
        # "tradeId": record.get("tradeId"),
        "OutTs": record.get("OutTs"),
        "ActTs": record.get("ActTs")
    }

def flatten_quote(record):
    """Flatten a quote (Q) message."""
    bid = record.get("bid", {})
    ask = record.get("ask", {})
    return {
        "exp": record.get("exp"),
        "sysTimestamp": record.get("header", {}).get("sysTimestamp"),
        "bid_price": bid.get("p"),
        "bid_quantity": bid.get("q"),
        "bid_c": bid.get("c"),
        "ask_price": ask.get("p"),
        "ask_quantity": ask.get("q"),
        "ask_c": ask.get("c"),
        "Seq": record.get("Seq"),
        "ActTs": record.get("ActTs"),
        "OutTs": record.get("OutTs")
    }

def flatten_snapshot(record):
    """Flatten a snapshot (S) message."""
    rows = []
    exp = record.get("exp")
    sysTimestamp = record.get("header", {}).get("sysTimestamp")
    snapshot_timestamp = record.get("timestamp")
    
    # Process bids: mark these rows with order_type "bid"
    for order in record.get("bids", []):
        row = {
            "exp": exp,
            "sysTimestamp": sysTimestamp,
            "snapshot_timestamp": snapshot_timestamp,
            "order_type": "bid",
            "price": order.get("p"),
            "quantity": order.get("q")
        }
        rows.append(row)
    
    # Process asks: mark these rows with order_type "ask"
    for order in record.get("asks", []):
        row = {
            "exp": exp,
            "sysTimestamp": sysTimestamp,
            "snapshot_timestamp": snapshot_timestamp,
            "order_type": "ask",
            "price": order.get("p"),
            "quantity": order.get("q")
        }
        rows.append(row)
    
    return rows

def segregate_data(json_file_path):
    """
    Read the file line-by-line and segregate the records by message type:
    trades (T), quotes (Q), snapshots (S).
    """
    trades = []
    quotes = []
    snapshots = []
    
    with open(json_file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                data = json.loads(line)
                msg_type = data.get("header", {}).get("type")
                if msg_type == "T":
                    trades.append(data)
                elif msg_type == "Q":
                    quotes.append(data)
                elif msg_type == "S":
                    snapshots.append(data)
                else:
                    print(f"Unknown message type: {msg_type}")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue
    
    return trades, quotes, snapshots

def process_records(trades, quotes, snapshots):
    """Flatten and convert each type of message to a DataFrame."""
    # Flatten trade messages
    flat_trades = [flatten_trade(rec) for rec in trades]
    df_trades = pd.DataFrame(flat_trades)
    
    # Flatten quote messages
    flat_quotes = [flatten_quote(rec) for rec in quotes]
    df_quotes = pd.DataFrame(flat_quotes)
    
    # Flatten snapshot messages:
    flat_snapshot_rows = []
    for rec in snapshots:
        flat_snapshot_rows.extend(flatten_snapshot(rec))
    df_snapshots = pd.DataFrame(flat_snapshot_rows)
    
    return df_trades, df_quotes, df_snapshots

def flatten_data(df):
    """Flatten the data."""
    df_trades = df[df['msg_type'] == 'T']
    df_quotes = df[df['msg_type'] == 'Q']
    df_snapshots = df[df['msg_type'] == 'S']

    df_trades = df_trades.apply(flatten_trade, axis=1, result_type='expand')
    df_quotes = df_quotes.apply(flatten_quote, axis=1, result_type='expand')
    df_snapshots = df_snapshots.apply(flatten_snapshot, axis=1, result_type='expand')

    return df_trades, df_quotes, df_snapshots

def convert_df_timestamps(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col].astype(float), unit='ns', errors='coerce')
    return df


In [ ]:
def main():
    # Path to the top-level "BINANCE_SPOT" folder that contains subfolders (e.g. 000002, 230000, etc.)
    base_dir = "C:/Users/choud/.ipython/Downloads/Sample1/OKEX_SPOT"
    
    # Create lists to hold partial DataFrames from each data.json
    all_trades = []
    all_quotes = []
    all_snapshots = []
    
    # Loop over each subfolder in BINANCE_SPOT (e.g. 000002, 230000, etc.)
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        
        if os.path.isdir(folder_path):
            # Inside each numeric folder, there's a BTC folder containing data.json
            btc_folder = os.path.join(folder_path, "SOL")
            if os.path.isdir(btc_folder):
                data_file = os.path.join(btc_folder, "data.json")
                if os.path.isfile(data_file):
                    print(f"Processing: {data_file}")
                    
                    # 1) Segregate data
                    trades, quotes, snapshots = segregate_data(data_file)
                    
                    # 2) Flatten into DataFrames
                    df_trades, df_quotes, df_snapshots = process_records(trades, quotes, snapshots)
                    
                    # 3) Convert timestamps
                    df_trades = convert_df_timestamps(df_trades, ["sysTimestamp", "ActTs", "OutTs"])
                    df_quotes = convert_df_timestamps(df_quotes, ["sysTimestamp", "ActTs", "OutTs"])
                    df_snapshots = convert_df_timestamps(df_snapshots, ["sysTimestamp", "snapshot_timestamp"])
                    
                    # 4) Append to global lists
                    all_trades.append(df_trades)
                    all_quotes.append(df_quotes)
                    all_snapshots.append(df_snapshots)
    
    # Concatenate all partial DataFrames into final ones
    final_trades = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame()
    final_quotes = pd.concat(all_quotes, ignore_index=True) if all_quotes else pd.DataFrame()
    final_snapshots = pd.concat(all_snapshots, ignore_index=True) if all_snapshots else pd.DataFrame()
    
    # Print or save
    print("\n--- Final Trades DataFrame ---")
    print(final_trades.head())
    
    print("\n--- Final Quotes DataFrame ---")
    print(final_quotes.head())
    
    print("\n--- Final Snapshots DataFrame ---")
    print(final_snapshots.head())
    
    # Save to CSV if desired
    final_trades.to_csv("trades_SOL_OKEX.csv", index=False)
    final_quotes.to_csv("quotes_SOL_OKEX.csv", index=False)
    final_snapshots.to_csv("snapshots_SOL_OKEX.csv", index=False)
    print("\nAll data saved to trades.csv, quotes.csv, snapshots.csv")

if __name__ == "__main__":
    main()